In [1]:
from import_data_encoded_knn import  *
from sequence_distance_matrix import *

from scipy.spatial.distance import cdist

def hamming_matrix_c(XA, XB, cat_features):
    return cdist(XA[cat_features], XB[cat_features], metric = "hamming")

def euclidean_matrix_c(XA, XB, num_features):
    return cdist(XA[num_features], XB[num_features], metric = "euclidean")

def basic_matrix_c(XA, XB, cat_features = [], num_features = [], a_ham = 0.0016102620275609393):
    return a_ham * hamming_matrix_c(XA, XB, cat_features) + euclidean_matrix_c(XA, XB, num_features)


X_try, X_train, X_test, y_train, y_test, len_X_train = import_data_encoded(encoding = 'binary')

categorical = ['ring_number', "exposure_type", "conc1_type","species",'tripleBond', 'obs_duration_mean', 'doubleBond',
    'alone_atom_number', 'class', 'tax_order', 'family', 'genus', 'oh_count']

non_categorical =[ 'atom_number', 'bonds_number', 'Mol', 'MorganDensity', 'LogP']

In [2]:
dist_matr_train = basic_matrix_c(X_train,X_train, categorical, non_categorical)
print('train fatto')
dist_matr_test = basic_matrix_c(X_test, X_train, categorical, non_categorical)
print('test fatto')

train fatto
test fatto


In [3]:
dist_matr_train = pd.DataFrame(dist_matr_train)
dist_matr_test = pd.DataFrame(dist_matr_test)

# Per allenare il KNN
dist_matr_train_0 = dist_matr_train.iloc[y_train == 0, y_train == 0]
dist_matr_train_1 = dist_matr_train.iloc[y_train == 1, y_train == 1]

# Per trovare i vicini train per gli esperimenti train --> df_rasar_train
dist_matr_train_train_0 = dist_matr_train.iloc[:,y_train == 0]
dist_matr_train_train_1 = dist_matr_train.iloc[:,y_train == 1]

# Per trovare i vicini train per gli esperimenti test --> df_rasar_test
dist_matr_test_train_0 = dist_matr_test.iloc[:, y_train == 0]
dist_matr_test_train_1 = dist_matr_test.iloc[:, y_train == 1]

# DF test

In [4]:
knn0 = KNeighborsClassifier(metric = 'precomputed', n_neighbors=1, n_jobs=-2, leaf_size=80)
knn0.fit(dist_matr_train_0, y_train[y_train == 0])

neigh0 = knn0.kneighbors(dist_matr_test_train_0, return_distance = True)

# con questi indici faccio X_train.loc[ idx ] per scoprire chi è il vicino dell'esperimento test
idx_neigh_0 = pd.DataFrame(neigh0[1])[0].apply(lambda x: X_train.iloc[y_train==0].iloc[x].name)

In [5]:
knn1 = KNeighborsClassifier(metric = 'precomputed', n_neighbors=1, n_jobs=-2, leaf_size=80)
knn1.fit(dist_matr_train_1, y_train[y_train == 1])

neigh1 = knn1.kneighbors(dist_matr_test_train_1, return_distance = True)

idx_neigh_1 = pd.DataFrame(neigh1[1])[0].apply(lambda x: X_train.iloc[y_train==1].iloc[x].name)

In [15]:
df_rasar_test = pd.DataFrame({'idx_test': X_test.index.values,
                  'label_test': y_test,
                  'idx_neigh0': idx_neigh_0.values,
                  'dist_neigh0': neigh0[0].ravel(),
                  'idx_neigh1': idx_neigh_1.values,
                  'dist_neigh1': neigh1[0].ravel()})
df_rasar_test

ValueError: arrays must all be same length

# DF Train

Qui si fa il confronto train vs train. 

Si allena il KNN con train0 e si vede con tutto il train, MA così facendo i train testati di classe 0 trovano se stessi come vicino. Quindi prendendo un 2-NN, prendo il secondo vicino che non è se stesso.

In [7]:
from help_rasar import *

In [8]:
knn0 = KNeighborsClassifier(metric = 'precomputed', n_jobs = -2, leaf_size = 80,
                            n_neighbors = 2)

knn0.fit(dist_matr_train_0, y_train[y_train == 0])

neigh0 = knn0.kneighbors(dist_matr_train_train_0, return_distance = True)

idx_neigh0, dist0 = right_neighbor(neigh0, X_train, y_train, 0)

In [9]:
knn1 = KNeighborsClassifier(metric = 'precomputed', n_jobs = -2, leaf_size = 80,
                            n_neighbors=2)

knn1.fit(dist_matr_train_1, y_train[y_train == 1])

neigh1 = knn1.kneighbors(dist_matr_train_train_1, return_distance = True)

idx_neigh1, dist1 = right_neighbor(neigh1, X_train, y_train, 1)

In [10]:
df_rasar_train = pd.DataFrame({'idx_train': X_train.index.values,
                  'label_train': y_train,
                  'idx_neigh0': idx_neigh0.values,
                  'dist_neigh0': dist0,       
                  'idx_neigh1': idx_neigh1.values,
                  'dist_neigh1': dist1})
df_rasar_train

,idx_train,label_train,idx_neigh0,dist_neigh0,idx_neigh1,dist_neigh1
0,848,1,21779,0.243507,850,0.000124
1,5049,0,5696,0.000000,5666,0.000495
2,20921,1,22671,0.010224,20915,0.000495
3,4571,0,12898,0.000124,3404,0.000619
4,14276,0,14277,0.000495,20640,0.129566
...,...,...,...,...,...,...
15620,11964,1,11903,0.000495,11965,0.000124
15621,21575,0,21571,0.000248,20777,0.062839
15622,5390,1,21424,0.252208,5392,0.000248
15623,860,1,21779,0.243878,858,0.000124


# Supervised Step

In [17]:
df_rasar_train[['dist_neigh0','dist_neigh1']]
df_rasar_test[['dist_neigh_0','dist_neigh_1']]

,dist_neigh_0,dist_neigh_1
0,0.000124,0.000124
1,0.000495,0.000124
2,0.000124,0.000124
3,0.241111,0.000619
4,0.189612,0.000495
...,...,...
7692,0.000124,0.000743
7693,0.000248,0.094760
7694,0.000495,0.122188
7695,0.000248,0.000495


In [30]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

lrc = LogisticRegression(random_state=0, fit_intercept = False, penalty = 'elasticnet', solver = 'saga',
                        l1_ratio = 1)

lrc.fit(df_rasar_train[['dist_neigh0','dist_neigh1']], y_train)

y_pred = lrc.predict(df_rasar_test[['dist_neigh_0','dist_neigh_1']])

accuracy_score(y_pred, y_test)

0.8959334805768481

In [5]:
# Con dati senza duplicati
knn = KNeighborsClassifier(metric = 'precomputed', n_neighbors=1, n_jobs=-2, leaf_size=80)
knn.fit(dist_matr_train, y_train)

y_pred = knn.predict(dist_matr_test)

accuracy_score(y_pred,y_test)

0.8975559022360895